# Tutorial 6: Generate context-aware cell embedding

In [ ]:
import torch.multiprocessing as mp
import pickle
import os
from pathlib import Path
import numpy as np
import pandas as pd
import torch
from torch.utils.data import DataLoader, Dataset
from collections import defaultdict, OrderedDict
from tqdm import tqdm
from argparse import Namespace
import gc

from omics.constants import *
import os
import torch.utils.data as data
from copy import deepcopy
import random
from sklearn.preprocessing import LabelEncoder
from omics.constants import *
import h5py
from pytorch_lightning.callbacks import (EarlyStopping, LearningRateMonitor,
                                         ModelCheckpoint, Callback)
from math import exp
import random
import h5py
import hashlib
import anndata as ad
from omics.Pretrain.omics.pretrain_fold2 import Omics
# from omics.Finetune.finetune_fold import Omics
from pytorch_lightning import LightningModule, Trainer, seed_everything
from argparse import ArgumentParser
from collections import defaultdict
from scipy.spatial import cKDTree
from step0_spot_input_gen import load_spot_data_with_full_info
from step2_uncond_cell_emb_gen import run_multi_gpu_processing_with_filtered_data

## Initialize args and fix seed

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"

args = Namespace(
    ckpt_path="SpotFormer_checkpoint.ckpt",
    dataset_name="mop1_filtered",
    target_dataset="mop1_filtered",
    gene_pct=100,
    use_multi_gpu=True,
    num_gpus=2,
    radius=20,  
    max_points=20,
    linear_hidden_dim='256', 
    config="configs/bert_config.json", 
    seed=42,
    f=None,
    split_slice=None,
    target_width=6724,
    target_height=5885,
    x_min=None,
    x_max=None,
    y_min=None,
    y_max=None,
    sample_ratio=1.0,
    merge_threshold=0.5,
    percentile=70
)

print('ckpt_path:', args.ckpt_path)
print(f"Dataset: {args.dataset_name}, gene_pct: {args.gene_pct}")
print(f"Use multi-GPU: {args.use_multi_gpu}, num_gpus: {args.num_gpus}")

seed_everything(args.seed)

## Generate unconditional cell embeddings

In [ ]:
global_h5_path = f'cached_spot_data/spot_input_spot_all_{args.dataset_name}_{args.seed}_{args.gene_pct}_{args.radius}_{args.max_points}.h5'

if args.use_multi_gpu:
    print("Using Multi-GPU processing with H5 split")
    cell_embeddings = run_multi_gpu_processing_with_filtered_data(args, global_h5_path, num_gpus=args.num_gpus)
else:
    print("Single-GPU processing not implemented for filtered data yet")

## Task: missing gene imputation

In [ ]:
from argparse import Namespace
import datetime
import os
import torch
from pytorch_lightning import Trainer, seed_everything
from pytorch_lightning.callbacks import (EarlyStopping, LearningRateMonitor,
                                         ModelCheckpoint)
from pytorch_lightning.loggers import WandbLogger
from dateutil import tz

from missing_gene_imputation.gimVI import OptimizedFixedReferenceBasedGimVI, optimize_dataloader_args
from omics.datasets.cell_dataset6 import QueRefCellDataModule

BASE_DIR = os.path.dirname(os.path.abspath(__file__))

### Initialize arguments

In [ ]:

# Define arguments directly
args = Namespace(
    # Model dimensions (will be updated from data)
    embedding_dim=768,
    n_available_genes=2000,
    n_target_genes=1000,
    spatial_dim=2,
    
    # Reference-based parameters
    latent_dim=256,
    reference_latent_dim=256,
    similarity_metric="cosine",  # "cosine", "learned", "l2"
    temperature=0.1,
    top_k_references=50,
    
    # Input modality selection (only one should be True)
    use_gene_expression=True,
    use_cell_embedding=False,
    
    # Output activation strategy
    output_activation="softplus",  # "relu", "softplus", "exp", "sigmoid"
    use_batch_norm_output=True,
    min_expression_value=1e-6,
    
    # Loss function improvements
    loss_function="mse",  # "mse", "poisson", "nb", "rmse"
    use_log_space_loss=False,
    expression_scale_factor=1.0,
    
    # Decoder parameters
    hidden_dim=256,
    dropout=0.1,
    use_spatial_decoder=True,
    
    # Training strategy
    reconstruction_weight=1.0,
    similarity_weight=0.5,
    consistency_weight=0.1,
    
    # Reference processing
    reference_sampling_ratio=1.0,
    reference_clustering=True,
    n_reference_clusters=200,
    
    # Training parameters
    learning_rate=1e-3,
    weight_decay=0.01,
    warmup_epochs=10,
    batch_size=128,
    epochs=100,
    num_workers=8,
    
    # Optimization parameters
    metric_compute_frequency=20,
    val_metric_compute_frequency=5,
    memory_cleanup_frequency=200,
    log_frequency=10,
    enable_fast_loss=True,
    
    # Data parameters
    normalize_embeddings=True,
    log_transform_counts=True,
    pin_memory=False,
    reference_ratio=0.7,
    reference_split_seed=42,
    
    # Evaluation parameters
    evaluation_output_dir="./optimized_reference_gimvi_evaluation",
    save_detailed_reports=True,
    save_anndata=True,
    compute_gene_level_metrics=True,
    
    # Experiment settings
    experiment_name="optimized_reference_based_gimvi",
    seed=42,
    dataset_name="mop1_filtered",  # "mop1_filtered", "seqfish", "AD_9494_9498"
    dataset_name2=None,
    fold="fold_1",
    data_pct=50,
    mode="test",  # "test" or "train_test"
    
    # Checkpoint
    ckpt_path=None,
    
    # Trainer settings
    accelerator=None,
    devices=None,
    deterministic=False,
    precision=32,
    gradient_clip_val=1.0,
    enable_progress_bar=True,
    enable_model_summary=False,
    sync_batchnorm=False,
    detect_anomaly=False,
)

# Update max_epochs
args.max_epochs = args.epochs


### Dataloader

In [ ]:
# Apply dataloader optimizations
print("Applying dataloader optimizations...")
args = optimize_dataloader_args(args)


# Data paths
if args.dataset_name == 'mop1_filtered':
    adata_path = f'cell_embeddings/processed_adata_mop1_filtered_42_{args.data_pct}.h5ad'
    adata_path2 = None
elif args.dataset_name == 'seqfish':
    adata_path = f'cell_embeddings/processed_adata_{args.dataset_name}_{args.seed}_{args.data_pct}.h5ad'
    adata_path2 = None
elif args.dataset_name == 'AD_9494_9498':
    adata_path = f'cell_embeddings/9494_adata_{args.data_pct}pct.h5ad'
    adata_path2 = f'cell_embeddings/9498_adata_{args.data_pct}pct.h5ad'

# Data module
datamodule = QueRefCellDataModule(
    adata_path=adata_path,
    adata_path2=adata_path2,
    fold_name=args.fold,
    batch_size=args.batch_size,
    num_workers=args.num_workers,
    normalize_embeddings=args.normalize_embeddings,
    log_transform_counts=args.log_transform_counts,
    pin_memory=args.pin_memory,
    reference_ratio=args.reference_ratio,
    k_neighbors=50,
    reference_split_seed=args.reference_split_seed
)

# Setup data module
datamodule.setup()
datamodule.dataset_name = args.dataset_name
datamodule.seed = args.seed
datamodule.data_pct = args.data_pct

# Get data dimensions
dims = datamodule.get_dimensions()

# Update model parameters
args.embedding_dim = dims['embedding_dim']
args.n_available_genes = dims['n_available_genes']
args.n_target_genes = dims['n_target_genes']
args.spatial_dim = dims['spatial_dim']

print(f"Optimized Reference gimVI dimensions:")
print(f"  embedding_dim: {args.embedding_dim}")
print(f"  n_available_genes: {args.n_available_genes}")
print(f"  n_target_genes: {args.n_target_genes}")
print(f"Optimization parameters:")
print(f"  Train metric frequency: {args.metric_compute_frequency}")
print(f"  Val metric frequency: {args.val_metric_compute_frequency}")
print(f"  Memory cleanup frequency: {args.memory_cleanup_frequency}")
print(f"  Log frequency: {args.log_frequency}")
print(f"  Fast loss: {args.enable_fast_loss}")

### Configure model training

In [ ]:
# Update max_epochs
args.max_epochs = args.epochs

# GPU Configuration
print("=== GPU Configuration ===")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device count: {torch.cuda.device_count()}")
    print(f"Current CUDA device: {torch.cuda.current_device()}")
    print(f"CUDA device name: {torch.cuda.get_device_name()}")
    
    if args.accelerator is None:
        args.accelerator = "gpu"
    if args.devices is None:
        args.devices = 1
        
    print(f"Trainer will use: accelerator={args.accelerator}, devices={args.devices}")
else:
    print("CUDA not available, using CPU")
    args.accelerator = "cpu"
    args.devices = 1
print("========================")

# Set random seed
seed_everything(args.seed)

### Training

In [ ]:
# Setup callbacks and logger
now = datetime.datetime.now(tz.tzlocal())
extension = now.strftime("%Y_%m_%d_%H_%M_%S")
ckpt_dir = os.path.join(
    BASE_DIR, f"../../../data/ckpts/optimized_reference_gimvi/{args.experiment_name}_{args.dataset_name}_{extension}")
os.makedirs(ckpt_dir, exist_ok=True)

# Fixed callbacks - use val_loss instead of val_pearson as primary monitoring metric
callbacks = [
    LearningRateMonitor(logging_interval="step"),
    ModelCheckpoint(
        monitor="val_loss",  # Changed to val_loss for better stability
        dirpath=ckpt_dir,
        save_last=True, 
        mode="min",  # Changed to min mode
        save_top_k=1
    ),
    EarlyStopping(
        monitor="val_loss",  # Changed to val_loss
        min_delta=0.001,
        patience=20,
        verbose=False, 
        mode="min"  # Changed to min mode
    )
]

logger_dir = os.path.join(BASE_DIR, f"../../../data")
os.makedirs(logger_dir, exist_ok=True)
wandb_logger = WandbLogger(
    project=f"Optimized_Reference_Based_gimVI", 
    save_dir=logger_dir, 
    name=f"{args.experiment_name}_{args.dataset_name}_{args.data_pct}_{args.fold}_{extension}"
)

# Optimized trainer configuration
trainer = Trainer(
    accelerator=args.accelerator,
    devices=args.devices,
    max_epochs=args.max_epochs,
    deterministic=args.deterministic,
    callbacks=callbacks,
    logger=wandb_logger,
    precision=args.precision,
    gradient_clip_val=args.gradient_clip_val,
    enable_progress_bar=args.enable_progress_bar,
    log_every_n_steps=args.log_frequency * 5,  # Reduced logging frequency
    enable_model_summary=args.enable_model_summary,
    sync_batchnorm=args.sync_batchnorm,
    detect_anomaly=args.detect_anomaly,
)

# Set trainer metadata
trainer.dataset_name = args.dataset_name
trainer.seed = args.seed
trainer.data_pct = args.data_pct

# Save checkpoint flag
is_save_ckpt = True

if args.mode == "train_test":
    # Training
    print("Training Optimized Reference-based gimVI...")
    trainer.fit(model, datamodule=datamodule)
    
    if is_save_ckpt:
        best_ckpt_path = os.path.join(ckpt_dir, "best_ckpts.yaml")
        callbacks[1].to_yaml(filepath=best_ckpt_path)
    
    # Load best model for testing
    args.ckpt_path = callbacks[1].best_model_path
    if args.ckpt_path:
        model = OptimizedFixedReferenceBasedGimVI.load_from_checkpoint(args.ckpt_path, **vars(args))

# Testing
print("Testing Optimized Reference-based gimVI...")
test_results = trainer.test(model, datamodule=datamodule)
print("Optimized Reference-based gimVI experiment completed!")
print(f"Test results: {test_results}")

## Task: cell type classification

### Initialize arguments

In [ ]:
from cell_type_classification.cellplm import *
from argparse import Namespace

# Define arguments directly
args = Namespace(
    # Pretrain settings
    pretrain_version='20230926_85M',
    pretrain_directory='ckpts/CellPLM',
    
    # Data and environment
    dataset_name='mop1_filtered',  # Required
    dataset_name2=None,
    fold='fold_1',
    data_pct=50,  # Required
    seed=42,
    label_type='class',  # 'class' or 'subclass'
    
    # Logging and saving
    evaluation_dir='./cellplm_evaluation',
    save_detailed_reports=True,  # Required
    save_anndata=True,  # Required
    compute_gene_level_metrics=True,  # Required
    
    # Mode control
    mode='test',  # 'test' or 'train_test'
    
    # Fusion related parameters
    enable_fusion=True,
    fusion_type='concat',  # 'concat', 'add', 'gated', 'attention'
    fusion_dropout=0.2,
)



### Data loading

In [ ]:
if args.dataset_name == 'mop1_filtered':
    data_path = f'/media/dang/Omics/omics/baseline/imputation/cell_embeddings_with_GT/processed_adata_mop1_filtered_42_{args.data_pct}.h5ad'
    data_path2 = None
else:
    data_path = f'/media/dang/Omics/omics/baseline/annotation/cell_embeddings_with_GT/processed_adata_{args.dataset_name}_{args.seed}_{args.data_pct}.h5ad'
    data_path2 = f'/media/dang/Omics/omics/baseline/annotation/cell_embeddings_with_GT/processed_adata_{args.dataset_name2}_{args.seed}_{args.data_pct}.h5ad'

# Load downstream dataset
data = load_data(data_path, data_path2, args.label_type, args.fold, args.seed)

### Model configuration

In [ ]:
# Overwrite parts of the default config
pipeline_config = CellTypeAnnotationDefaultPipelineConfig.copy()

model_config = CellTypeAnnotationDefaultModelConfig.copy()
model_config['out_dim'] = data.obs['celltype'].nunique()

# Fine-tuning the model (freeze encoder, only train decoder)
pipeline_config = CellTypeAnnotationDefaultPipelineConfig.copy()

model_config = CellTypeAnnotationDefaultModelConfig.copy()
model_config['out_dim'] = data.obs['celltype'].nunique()
pipeline_config, model_config

model_config['enable_cell_embedding'] = True
model_config['post_latent_dim'] = 768
model_config['enable_fusion'] = args.enable_fusion
model_config['fusion_type'] = args.fusion_type
model_config['fusion_dropout'] = args.fusion_dropout
model_config['enc_hid'] = 217 if args.dataset_name == 'mop1_filtered' else 1000

pipeline = CellTypeAnnotationPipeline(pretrain_prefix=args.pretrain_version,  # Specify the pretrain checkpoint to load
                                    overwrite_config=model_config,  # This is for overwriting part of the pretrain config
                                    pretrain_directory=args.pretrain_directory)
pipeline.model

### Model training and evaluation

In [ ]:
if args.mode == 'train_test':
    pipeline.fit(data,  # An AnnData object
            pipeline_config,  # The config dictionary we created previously, optional
            split_field='split',  # Specify a column in .obs that contains split information
            train_split='train',
            valid_split='valid',
            label_fields=['celltype'],  # Specify a column in .obs that contains cell type labels
            enable_fusion=args.enable_fusion)  # Enable fusion functionality

# Inference and evaluation
if args.mode == 'test':
    pipeline.fitted = True  # Although it doesn't support not fitted, we can force set to True since checkpoint is loaded

pipeline.predict(
                data,  # An AnnData object
                pipeline_config,  # The config dictionary we created previously, optional
                label_fields=['celltype'],
                enable_fusion=args.enable_fusion  # Enable fusion functionality
            )

scores = pipeline.score(data,  # An AnnData object
            pipeline_config,  # The config dictionary we created previously, optional
            split_field='split',  # Specify a column in .obs to specify train and valid split, optional
            target_split='test',  # Specify a target split to predict, optional
            label_fields=['celltype'],  # Specify a column in .obs that contains cell type labels
            dataset_name=args.dataset_name,
            fold_name=args.fold,
            data_pct=args.data_pct,
            seed=args.seed,
            label_type=args.label_type,
            evaluation_output_dir=args.evaluation_dir,
            save_detailed_reports=args.save_detailed_reports,
            save_anndata=args.save_anndata)  

print(scores)

### AUC evaluation

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import matplotlib as mpl
mpl.rcParams['pdf.fonttype'] = 42
plt.rcParams['savefig.dpi'] = 300 
plt.rcParams['figure.dpi'] = 300 

import scipy.stats as stats
import mpl_toolkits.axisartist as axisartist
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
import scanpy as sc
from matplotlib.colors import LinearSegmentedColormap

baseline_path = "cellplm_evaluation/cellPLM_mop1_filtered_fold_1_42_20250819_215447_50_subclass.h5ad"
enhanced_path = "cellplm_enhanced_evaluation/cellPLM_mop1_filtered_fold_1_42_20250825_000654_50_subclass.h5ad"

from cell_type_classification.eval import calculate_auc_comparison

print("🚀 Start calculating AUC...")
auc_df = calculate_auc_comparison(baseline_path, enhanced_path)

print("\n✅ AUC calculation completed!")
print("📁 Generated file: auc_results.csv")
    

### Figure generation

In [ ]:
import matplotlib.pyplot as plt
from cell_type_classification.fig_gen import create_heatmaps_single_cell_confidence_seaborn

plt.rcParams.update({
    'font.size': 10,
    'axes.labelsize': 12,
    'axes.titlesize': 14,
    'legend.fontsize': 9,
    'figure.dpi': 100
})

baseline_path = "cellplm_evaluation/cellPLM_mop1_filtered_fold_1_42_20250819_215447_50_subclass.h5ad"
enhanced_path = "cellplm_enhanced_evaluation/cellPLM_mop1_filtered_fold_1_42_20250825_000654_50_subclass.h5ad"

print("Starting single-cell confidence heatmap generation (using seaborn template)...")
baseline_fig, enhanced_fig = create_heatmaps_single_cell_confidence_seaborn(
    baseline_path, enhanced_path, max_cells=2000
)

print("\nAll single-cell confidence heatmap generation complete!")

## Generate conditional cell embeddings

### match spot embeddings into cell

In [ ]:
from step3_cell_matching import match_cells_by_coordinates
args = Namespace(
    dataset_id="9494",
    output_dir="AD_cell_embeddings",
    data_name=None
)

In [ ]:
adata_path = f'AD/2022-09-16-Hu-AD-stardist-scaled.h5ad'

print("=== Reading Cell Embeddings AnnData ===")

# Load file
adata_cells = ad.read_h5ad(adata_path)

adata_9494 = adata_cells[adata_cells.obs['sample'] == f'ADmouse_{args.dataset_id}']

# Read cell embeddings AnnData file
adata_emb_path = f'spot_input_spot_all_AD_2766g_m{args.dataset_id}_42_100_20_20{args.data_name}_cells.h5ad'

# Load file
adata_emb = ad.read_h5ad(adata_emb_path)

# Usage example
matched_adata = match_cells_by_coordinates(adata_9494, adata_emb, tolerance=20)

# Ensure correct data types for all string columns before saving
matched_adata.obs['cell_orig_id'] = matched_adata.obs['cell_orig_id'].astype(str)
matched_adata.obs['cell_total_id'] = matched_adata.obs['cell_total_id'].astype(str)

# Create output directory
os.makedirs(args.output_dir, exist_ok=True)

# Save matching results
matched_adata.write_h5ad(f'{args.output_dir}/AD_cell_matching_m{args.dataset_id}{args.data_name}.h5ad')

# View matching results
matched_count = (matched_adata.obs['cell_orig_id'] != 'unmatched').sum()
print(f"Successfully matched cells: {matched_count}")
print(f"Total cells: {len(matched_adata.obs)}")
print(f"Matching rate: {matched_count / len(matched_adata.obs):.2%}")
print("\nMatching results example:")
print(matched_adata.obs[['center_x', 'center_y', 'cell_orig_id', 'cell_total_id']].head())

# View successfully matched cells
matched_cells = matched_adata.obs[matched_adata.obs['cell_orig_id'] != 'unmatched']
if len(matched_cells) > 0:
    print("\nSuccessfully matched cells example:")
    print(matched_cells[['center_x', 'center_y', 'cell_orig_id', 'cell_total_id']].head())

### Train conditional aggregation and cell embedding

In [ ]:
from step4_cond_cell_emb_gen import run_train_and_cell_emb_gen

run_train_and_cell_emb_gen()